# CM[5] Kaggle Competition Group38

In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("dkmacovid_kaggletest_features.csv")

In [4]:
df.head(5)

,Id,Day,State ID,State,Lat,Long_,Active,Incident_Rate,Total_Test_Results,Case_Fatality_Ratio,Testing_Rate,Resident Population 2020 Census,Population Density 2020 Census,Density Rank 2020 Census,SexRatio
0,0,2,14,Illinois,40.3495,-88.9861,957138,7697.015291,13436652,1.867428,106035.6834,"12,812,508",230.8,14,97
1,5,3,14,Illinois,40.3495,-88.9861,961499,7732.282519,13482117,1.869933,106394.4716,"12,812,508",230.8,14,97
2,10,4,14,Illinois,40.3495,-88.9861,966468,7772.205747,13530371,1.869466,106775.2693,"12,812,508",230.8,14,97
3,15,5,14,Illinois,40.3495,-88.9861,973157,7826.175891,13617454,1.871700,107462.4870,"12,812,508",230.8,14,97
4,20,6,14,Illinois,40.3495,-88.9861,980553,7885.906848,13698428,1.874835,108101.4954,"12,812,508",230.8,14,97


In [5]:
df['Resident Population 2020 Census'] = df['Resident Population 2020 Census'].str.replace(',','').astype('int')

In [6]:
df_gstate = df.groupby('State')
for key,value in df_gstate:
    groups = df_gstate.get_group(key)
    temp = groups.iloc[:,4:]
    for columns in temp:
        Q1 = np.percentile(temp[columns],25)
        Q3 = np.percentile(temp[columns],75)
        IQR = Q3 - Q1
        right_limit = Q3 + 1.5*IQR
        left_limit = Q1 - 1.5*IQR
        outlier_right_index = groups[groups[columns] > right_limit][columns].index
        outlier_left_index = groups[groups[columns] < left_limit][columns].index
        n_outliers = len(outlier_right_index) + len(outlier_left_index)
        if(n_outliers > 0):
            print(key,columns,n_outliers)
            df.loc[outlier_right_index,columns] = right_limit
            df.loc[outlier_left_index,columns] = left_limit

Iowa Case_Fatality_Ratio 1
Washington Case_Fatality_Ratio 2


In [7]:
X_test = df.iloc[:,4:]

In [8]:
X_test = (X_test - X_test.mean())/ X_test.std()

In [9]:
X_test

,Lat,Long_,Active,Incident_Rate,Total_Test_Results,Case_Fatality_Ratio,Testing_Rate,Resident Population 2020 Census,Population Density 2020 Census,Density Rank 2020 Census,SexRatio
0,0.185758,0.685045,1.624921,-0.034555,0.871610,1.714293,1.248503,0.093862,1.600344,-1.387770,-0.557150
1,0.185758,0.685045,1.636807,-0.019067,0.879419,1.725587,1.261108,0.093862,1.600344,-1.387770,-0.557150
2,0.185758,0.685045,1.650350,-0.001535,0.887708,1.723482,1.274487,0.093862,1.600344,-1.387770,-0.557150
3,0.185758,0.685045,1.668580,0.022167,0.902667,1.733551,1.298632,0.093862,1.600344,-1.387770,-0.557150
4,0.185758,0.685045,1.688737,0.048398,0.916576,1.747687,1.321083,0.093862,1.600344,-1.387770,-0.557150
...,...,...,...,...,...,...,...,...,...,...,...
145,1.449287,-1.903393,-0.163129,-1.654112,-0.672606,-0.486315,-0.425243,-0.463803,-0.351668,-0.102798,1.114301
146,1.449287,-1.903393,-0.156348,-1.639579,-0.668592,-0.490357,-0.414460,-0.463803,-0.351668,-0.102798,1.114301
147,1.449287,-1.903393,-0.151034,-1.628091,-0.664255,-0.469195,-0.402811,-0.463803,-0.351668,-0.102798,1.114301
148,1.449287,-1.903393,-0.146139,-1.617734,-0.659315,-0.505139,-0.389542,-0.463803,-0.351668,-0.102798,1.114301


## Preprocessing Completed

In [10]:
x_test = np.array(X_test)
x_test1 = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
%store -r x_train1 y_train x_train

### MLP Model

### Label: Recovered

In [11]:
tf.keras.backend.clear_session()
lstm_recovered = tf.keras.Sequential([
                                tf.keras.Input(shape=(11,)),
                                tf.keras.layers.Dense(20,activation = 'relu'),
                                tf.keras.layers.Dense(20,activation = 'relu'),
                                tf.keras.layers.Dense(1,activation = 'sigmoid')])
lstm_recovered.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                240       
_________________________________________________________________
dense_1 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 681
Trainable params: 681
Non-trainable params: 0
_________________________________________________________________


In [12]:
lstm_recovered.compile(optimizer='adam',
              loss='BinaryCrossentropy',
              metrics=['accuracy'])

In [13]:
lstm_recovered.fit(x_train,y_train.iloc[:,-1],epochs=200,validation_split=0.2)

Epoch 1/200
7/7 [==============================] - 0s 22ms/step - loss: 0.7460 - accuracy: 0.4000 - val_loss: 0.6977 - val_accuracy: 0.4643
Epoch 2/200
7/7 [==============================] - 0s 5ms/step - loss: 0.7133 - accuracy: 0.4273 - val_loss: 0.6848 - val_accuracy: 0.5000
Epoch 3/200
7/7 [==============================] - 0s 4ms/step - loss: 0.6907 - accuracy: 0.4818 - val_loss: 0.6759 - val_accuracy: 0.5714
Epoch 4/200
7/7 [==============================] - 0s 4ms/step - loss: 0.6694 - accuracy: 0.5455 - val_loss: 0.6688 - val_accuracy: 0.5893
Epoch 5/200
7/7 [==============================] - 0s 4ms/step - loss: 0.6533 - accuracy: 0.6636 - val_loss: 0.6629 - val_accuracy: 0.6786
Epoch 6/200
7/7 [==============================] - 0s 3ms/step - loss: 0.6389 - accuracy: 0.7045 - val_loss: 0.6580 - val_accuracy: 0.6786
Epoch 7/200
7/7 [==============================] - 0s 4ms/step - loss: 0.6270 - accuracy: 0.7227 - val_loss: 0.6534 - val_accuracy: 0.6607
Epoch 8/200
7/7 [=========

7/7 [==============================] - 0s 4ms/step - loss: 0.2813 - accuracy: 0.9227 - val_loss: 0.3525 - val_accuracy: 0.8750
Epoch 60/200
7/7 [==============================] - 0s 4ms/step - loss: 0.2763 - accuracy: 0.9318 - val_loss: 0.3481 - val_accuracy: 0.8750
Epoch 61/200
7/7 [==============================] - 0s 5ms/step - loss: 0.2722 - accuracy: 0.9273 - val_loss: 0.3400 - val_accuracy: 0.8750
Epoch 62/200
7/7 [==============================] - 0s 4ms/step - loss: 0.2690 - accuracy: 0.9318 - val_loss: 0.3367 - val_accuracy: 0.8750
Epoch 63/200
7/7 [==============================] - 0s 4ms/step - loss: 0.2649 - accuracy: 0.9364 - val_loss: 0.3293 - val_accuracy: 0.8750
Epoch 64/200
7/7 [==============================] - 0s 4ms/step - loss: 0.2611 - accuracy: 0.9409 - val_loss: 0.3293 - val_accuracy: 0.8750
Epoch 65/200
7/7 [==============================] - 0s 4ms/step - loss: 0.2577 - accuracy: 0.9409 - val_loss: 0.3235 - val_accuracy: 0.8750
Epoch 66/200
7/7 [===============

7/7 [==============================] - 0s 4ms/step - loss: 0.1546 - accuracy: 0.9591 - val_loss: 0.2426 - val_accuracy: 0.9286
Epoch 118/200
7/7 [==============================] - 0s 3ms/step - loss: 0.1536 - accuracy: 0.9591 - val_loss: 0.2465 - val_accuracy: 0.9286
Epoch 119/200
7/7 [==============================] - 0s 5ms/step - loss: 0.1534 - accuracy: 0.9591 - val_loss: 0.2415 - val_accuracy: 0.9286
Epoch 120/200
7/7 [==============================] - 0s 3ms/step - loss: 0.1528 - accuracy: 0.9591 - val_loss: 0.2495 - val_accuracy: 0.9286
Epoch 121/200
7/7 [==============================] - 0s 3ms/step - loss: 0.1511 - accuracy: 0.9591 - val_loss: 0.2404 - val_accuracy: 0.9107
Epoch 122/200
7/7 [==============================] - 0s 3ms/step - loss: 0.1492 - accuracy: 0.9591 - val_loss: 0.2402 - val_accuracy: 0.9107
Epoch 123/200
7/7 [==============================] - 0s 4ms/step - loss: 0.1484 - accuracy: 0.9591 - val_loss: 0.2429 - val_accuracy: 0.9107
Epoch 124/200
7/7 [========

7/7 [==============================] - 0s 4ms/step - loss: 0.1152 - accuracy: 0.9636 - val_loss: 0.2432 - val_accuracy: 0.9107
Epoch 176/200
7/7 [==============================] - 0s 3ms/step - loss: 0.1146 - accuracy: 0.9636 - val_loss: 0.2443 - val_accuracy: 0.9107
Epoch 177/200
7/7 [==============================] - 0s 4ms/step - loss: 0.1146 - accuracy: 0.9636 - val_loss: 0.2468 - val_accuracy: 0.9107
Epoch 178/200
7/7 [==============================] - 0s 3ms/step - loss: 0.1138 - accuracy: 0.9636 - val_loss: 0.2445 - val_accuracy: 0.9107
Epoch 179/200
7/7 [==============================] - 0s 4ms/step - loss: 0.1149 - accuracy: 0.9636 - val_loss: 0.2354 - val_accuracy: 0.9107
Epoch 180/200
7/7 [==============================] - 0s 3ms/step - loss: 0.1134 - accuracy: 0.9636 - val_loss: 0.2385 - val_accuracy: 0.9107
Epoch 181/200
7/7 [==============================] - 0s 3ms/step - loss: 0.1128 - accuracy: 0.9636 - val_loss: 0.2461 - val_accuracy: 0.9107
Epoch 182/200
7/7 [========

In [14]:
predicted_recovered = lstm_recovered.predict(x_test) > 0.5

In [15]:
predicted_recovered.astype('int')

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

### Label: Deaths

In [16]:
pd_deaths = pd.read_csv('cleaned_deaths_coviddata.csv')
x_dtrain,y_dtrain = pd_deaths.iloc[:,:-1],pd_deaths.iloc[:,-1]

In [17]:
tf.keras.backend.clear_session()
lstm_deaths = tf.keras.Sequential([
                                 tf.keras.Input(shape=(11,)),
                                tf.keras.layers.Dense(40,activation = 'relu'),
                                tf.keras.layers.Dense(20,activation = 'relu'),
                                tf.keras.layers.Dense(1,activation = 'sigmoid')])
lstm_deaths.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 40)                480       
_________________________________________________________________
dense_1 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 1,321
Trainable params: 1,321
Non-trainable params: 0
_________________________________________________________________


In [18]:
lstm_deaths.compile(optimizer='adam',
              loss='BinaryCrossentropy',
              metrics=['accuracy'])

In [19]:
lstm_deaths.fit(x_dtrain,y_dtrain,epochs=190,validation_split=0.2)

Epoch 1/190
63/63 [==============================] - 0s 2ms/step - loss: 0.6263 - accuracy: 0.6477 - val_loss: 0.7398 - val_accuracy: 0.4639
Epoch 2/190
63/63 [==============================] - 0s 1ms/step - loss: 0.5328 - accuracy: 0.7211 - val_loss: 0.6553 - val_accuracy: 0.6606
Epoch 3/190
63/63 [==============================] - 0s 996us/step - loss: 0.4919 - accuracy: 0.7497 - val_loss: 0.6328 - val_accuracy: 0.6466
Epoch 4/190
63/63 [==============================] - 0s 1ms/step - loss: 0.4645 - accuracy: 0.7693 - val_loss: 0.6313 - val_accuracy: 0.6145
Epoch 5/190
63/63 [==============================] - 0s 1ms/step - loss: 0.4481 - accuracy: 0.7749 - val_loss: 0.6309 - val_accuracy: 0.6325
Epoch 6/190
63/63 [==============================] - 0s 1ms/step - loss: 0.4356 - accuracy: 0.7844 - val_loss: 0.6891 - val_accuracy: 0.5843
Epoch 7/190
63/63 [==============================] - 0s 1ms/step - loss: 0.4255 - accuracy: 0.7925 - val_loss: 0.5873 - val_accuracy: 0.7430
Epoch 8/190

63/63 [==============================] - 0s 1ms/step - loss: 0.3471 - accuracy: 0.8256 - val_loss: 0.4799 - val_accuracy: 0.8434
Epoch 59/190
63/63 [==============================] - 0s 1ms/step - loss: 0.3438 - accuracy: 0.8347 - val_loss: 0.4700 - val_accuracy: 0.8153
Epoch 60/190
63/63 [==============================] - 0s 1ms/step - loss: 0.3427 - accuracy: 0.8307 - val_loss: 0.3956 - val_accuracy: 0.8876
Epoch 61/190
63/63 [==============================] - 0s 1ms/step - loss: 0.3448 - accuracy: 0.8307 - val_loss: 0.5071 - val_accuracy: 0.8012
Epoch 62/190
63/63 [==============================] - 0s 1ms/step - loss: 0.3416 - accuracy: 0.8372 - val_loss: 0.4013 - val_accuracy: 0.8896
Epoch 63/190
63/63 [==============================] - 0s 1ms/step - loss: 0.3410 - accuracy: 0.8377 - val_loss: 0.5898 - val_accuracy: 0.7470
Epoch 64/190
63/63 [==============================] - 0s 2ms/step - loss: 0.3390 - accuracy: 0.8372 - val_loss: 0.4742 - val_accuracy: 0.8534
Epoch 65/190
63/63 

Epoch 116/190
63/63 [==============================] - 0s 1ms/step - loss: 0.3229 - accuracy: 0.8492 - val_loss: 0.4040 - val_accuracy: 0.8594
Epoch 117/190
63/63 [==============================] - 0s 1ms/step - loss: 0.3223 - accuracy: 0.8508 - val_loss: 0.4158 - val_accuracy: 0.8534
Epoch 118/190
63/63 [==============================] - 0s 1ms/step - loss: 0.3207 - accuracy: 0.8452 - val_loss: 0.4100 - val_accuracy: 0.8695
Epoch 119/190
63/63 [==============================] - 0s 1ms/step - loss: 0.3204 - accuracy: 0.8457 - val_loss: 0.4034 - val_accuracy: 0.8855
Epoch 120/190
63/63 [==============================] - 0s 1ms/step - loss: 0.3232 - accuracy: 0.8482 - val_loss: 0.4137 - val_accuracy: 0.8635
Epoch 121/190
63/63 [==============================] - 0s 1ms/step - loss: 0.3228 - accuracy: 0.8462 - val_loss: 0.3579 - val_accuracy: 0.9036
Epoch 122/190
63/63 [==============================] - 0s 1ms/step - loss: 0.3203 - accuracy: 0.8442 - val_loss: 0.4037 - val_accuracy: 0.8735

63/63 [==============================] - 0s 1ms/step - loss: 0.3084 - accuracy: 0.8467 - val_loss: 0.3996 - val_accuracy: 0.8855
Epoch 174/190
63/63 [==============================] - 0s 1ms/step - loss: 0.3077 - accuracy: 0.8528 - val_loss: 0.3919 - val_accuracy: 0.8795
Epoch 175/190
63/63 [==============================] - 0s 1ms/step - loss: 0.3087 - accuracy: 0.8452 - val_loss: 0.3929 - val_accuracy: 0.8574
Epoch 176/190
63/63 [==============================] - 0s 1ms/step - loss: 0.3118 - accuracy: 0.8518 - val_loss: 0.3828 - val_accuracy: 0.8655
Epoch 177/190
63/63 [==============================] - 0s 1ms/step - loss: 0.3098 - accuracy: 0.8472 - val_loss: 0.3293 - val_accuracy: 0.9116
Epoch 178/190
63/63 [==============================] - 0s 1ms/step - loss: 0.3065 - accuracy: 0.8497 - val_loss: 0.3446 - val_accuracy: 0.9116
Epoch 179/190
63/63 [==============================] - 0s 1ms/step - loss: 0.3077 - accuracy: 0.8508 - val_loss: 0.3798 - val_accuracy: 0.8655
Epoch 180/190

In [20]:
predicted_deaths = (lstm_deaths.predict(x_test) > 0.50).astype('int')

In [21]:
predicted_deaths

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

### Label : Confirmed

In [22]:
pd_confirmed = pd.read_csv('cleaned_confirmed_coviddata.csv')
x_ctrain,y_ctrain = pd_confirmed.iloc[:,:-1],pd_confirmed.iloc[:,-1]

In [23]:
tf.keras.backend.clear_session()
lstm_confirmed = tf.keras.Sequential([
                                tf.keras.Input(shape=(11,)),
                                tf.keras.layers.Dense(20,activation = 'relu'),
                                tf.keras.layers.Dense(20,activation = 'relu'),
                                tf.keras.layers.Dense(1,activation = 'sigmoid')])
lstm_confirmed.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                240       
_________________________________________________________________
dense_1 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 681
Trainable params: 681
Non-trainable params: 0
_________________________________________________________________


In [24]:
lstm_confirmed.compile(optimizer='adam',
              loss='BinaryCrossentropy',
              metrics=['accuracy'])

In [25]:
lstm_confirmed.fit(x_ctrain,y_ctrain,epochs=200,validation_split=0.2)

Epoch 1/200
44/44 [==============================] - 0s 3ms/step - loss: 0.6348 - accuracy: 0.6838 - val_loss: 0.7292 - val_accuracy: 0.3873
Epoch 2/200
44/44 [==============================] - 0s 1ms/step - loss: 0.5552 - accuracy: 0.7192 - val_loss: 0.7232 - val_accuracy: 0.3526
Epoch 3/200
44/44 [==============================] - 0s 1ms/step - loss: 0.4983 - accuracy: 0.7525 - val_loss: 0.6920 - val_accuracy: 0.5058
Epoch 4/200
44/44 [==============================] - 0s 1ms/step - loss: 0.4543 - accuracy: 0.7800 - val_loss: 0.6421 - val_accuracy: 0.5636
Epoch 5/200
44/44 [==============================] - 0s 1ms/step - loss: 0.4216 - accuracy: 0.8148 - val_loss: 0.5900 - val_accuracy: 0.6965
Epoch 6/200
44/44 [==============================] - 0s 2ms/step - loss: 0.3939 - accuracy: 0.8198 - val_loss: 0.5511 - val_accuracy: 0.7225
Epoch 7/200
44/44 [==============================] - 0s 1ms/step - loss: 0.3710 - accuracy: 0.8300 - val_loss: 0.5223 - val_accuracy: 0.7341
Epoch 8/200
4

Epoch 59/200
44/44 [==============================] - 0s 1ms/step - loss: 0.1692 - accuracy: 0.9414 - val_loss: 0.2076 - val_accuracy: 0.9335
Epoch 60/200
44/44 [==============================] - 0s 1ms/step - loss: 0.1687 - accuracy: 0.9414 - val_loss: 0.1916 - val_accuracy: 0.9393
Epoch 61/200
44/44 [==============================] - 0s 1ms/step - loss: 0.1694 - accuracy: 0.9407 - val_loss: 0.2131 - val_accuracy: 0.9335
Epoch 62/200
44/44 [==============================] - 0s 1ms/step - loss: 0.1684 - accuracy: 0.9392 - val_loss: 0.1864 - val_accuracy: 0.9393
Epoch 63/200
44/44 [==============================] - 0s 1ms/step - loss: 0.1673 - accuracy: 0.9399 - val_loss: 0.1950 - val_accuracy: 0.9393
Epoch 64/200
44/44 [==============================] - 0s 1ms/step - loss: 0.1663 - accuracy: 0.9407 - val_loss: 0.1851 - val_accuracy: 0.9422
Epoch 65/200
44/44 [==============================] - 0s 1ms/step - loss: 0.1692 - accuracy: 0.9392 - val_loss: 0.2093 - val_accuracy: 0.9249
Epoch 

44/44 [==============================] - 0s 1ms/step - loss: 0.1578 - accuracy: 0.9407 - val_loss: 0.1530 - val_accuracy: 0.9364
Epoch 117/200
44/44 [==============================] - 0s 1ms/step - loss: 0.1582 - accuracy: 0.9407 - val_loss: 0.1730 - val_accuracy: 0.9277
Epoch 118/200
44/44 [==============================] - 0s 1ms/step - loss: 0.1574 - accuracy: 0.9399 - val_loss: 0.1702 - val_accuracy: 0.9220
Epoch 119/200
44/44 [==============================] - 0s 1ms/step - loss: 0.1603 - accuracy: 0.9399 - val_loss: 0.1874 - val_accuracy: 0.9364
Epoch 120/200
44/44 [==============================] - 0s 1ms/step - loss: 0.1562 - accuracy: 0.9414 - val_loss: 0.1657 - val_accuracy: 0.9364
Epoch 121/200
44/44 [==============================] - 0s 2ms/step - loss: 0.1560 - accuracy: 0.9392 - val_loss: 0.1559 - val_accuracy: 0.9393
Epoch 122/200
44/44 [==============================] - 0s 1ms/step - loss: 0.1574 - accuracy: 0.9407 - val_loss: 0.1870 - val_accuracy: 0.9191
Epoch 123/200

44/44 [==============================] - 0s 1ms/step - loss: 0.1537 - accuracy: 0.9421 - val_loss: 0.1795 - val_accuracy: 0.9249
Epoch 174/200
44/44 [==============================] - 0s 1ms/step - loss: 0.1542 - accuracy: 0.9385 - val_loss: 0.1669 - val_accuracy: 0.9364
Epoch 175/200
44/44 [==============================] - 0s 1ms/step - loss: 0.1528 - accuracy: 0.9428 - val_loss: 0.1613 - val_accuracy: 0.9364
Epoch 176/200
44/44 [==============================] - 0s 1ms/step - loss: 0.1551 - accuracy: 0.9407 - val_loss: 0.1766 - val_accuracy: 0.9335
Epoch 177/200
44/44 [==============================] - 0s 1ms/step - loss: 0.1551 - accuracy: 0.9385 - val_loss: 0.1900 - val_accuracy: 0.9364
Epoch 178/200
44/44 [==============================] - 0s 1ms/step - loss: 0.1547 - accuracy: 0.9407 - val_loss: 0.1491 - val_accuracy: 0.9422
Epoch 179/200
44/44 [==============================] - 0s 1ms/step - loss: 0.1538 - accuracy: 0.9421 - val_loss: 0.1655 - val_accuracy: 0.9306
Epoch 180/200

In [26]:
predicted_confirmed = (lstm_confirmed.predict(x_test) > 0.50).astype('int')

In [27]:
new_df = pd.DataFrame()

In [28]:
new_df['Id'] = df.loc[:,'Id']
new_df['Confirmed'] = predicted_confirmed
new_df['Deaths'] = predicted_deaths
new_df['Recovered'] = predicted_recovered.astype('int')

In [29]:
new_df.to_csv("Kagglepred_assgn3.csv",index = False)